Import necessary libraries.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Run queries to get selected data from SQL into Pandas dataframe. 

In [2]:
engine = create_engine('sqlite:///D:\\Data\\DuvalProperties.sqlite')
con = engine.connect()
inv = con.execute(
    """
    SELECT
        RE, section, township, range, tile, mailing_address_1, mailing_address_2, city AS mail_city, 
        state AS mail_state, substr(zipcode, 1, 5) AS mail_zipcode, 
		subdivision_name, neighborhood, perc_capped, just_value, 
        ROUND((school_taxable - just_value) / just_value * 100,2) AS perc_school_deduction, 
		ROUND((county_taxable - just_value) / just_value * 100,2) AS perc_county_deduction, 
        tax_district, lot_sf,
		building, type_descr, style, class, quality, actual_yr_built, 
        date('now') - actual_yr_built AS age, effec_yr_built, date('now') - effec_yr_built AS effec_age, 
        value AS building_value, heated_sf
    FROM Parcel
    LEFT JOIN Building
    USING (RE)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    """
    )
housing_inventory = pd.DataFrame(inv.fetchall(),
                                columns=['RE', 'section', 'township', 'range', 'tile', 'mailing_address_1',
                                        'mailing_address_2', 'mail_city', 'mail_state', 'mail_zipcode',
                                        'subdivision_name', 'neighborhood', 'perc_capped', 'just_value',
                                        'perc_school_deduction', 'perc_county_deduction', 'tax_district',
                                        'lot_sf', 'building', 'type_descr', 'style', 'class', 'quality', 
                                        'actual_yr_built', 'age', 'effec_yr_built', 'effec_age',
                                        'building_value', 'heated_sf'])

aggregates = con.execute(
    """
    SELECT 
        RE, building, bldng_count, land_value, feature_count, avg_ft_grade, features_value, sub_ft_count,
        gross_sf, effec_sf, parcel_bldng_sf
    FROM
    (
    SELECT 
        RE, building, property_use, type_descr, IFNULL(bldng_count,0) AS bldng_count, 
        IFNULL(land_value,0) AS land_value, feature_count, avg_ft_grade, avg_ft_age, features_value, 
        sub_ft_count, heated_sf, gross_sf, effec_sf, parcel_bldng_sf
    FROM Parcel
    LEFT JOIN Building USING (RE)
    LEFT JOIN
    /* sub1 */
    (SELECT RE, COUNT(building) AS bldng_count
    FROM Building
    GROUP BY RE) AS sub1
    USING (RE)
    /* sub2 */
    LEFT JOIN
    (SELECT RE, SUM(land_val) AS land_value
    FROM Common
    GROUP BY RE) AS sub2
    USING (RE)
    /* sub3 */
    LEFT JOIN
    (SELECT RE, building, COUNT(*) AS feature_count, ROUND(AVG(grade),1) AS avg_ft_grade, ROUND(AVG(2021 - actual_yr_built),1) AS avg_ft_age, SUM(deprec_value) AS features_value
    FROM Feature
    WHERE actual_yr_built > 0
    GROUP BY RE, building) AS sub3
    USING (RE, building)
    /* sub4 */
    LEFT JOIN 
    (SELECT RE, building, COUNT(substructure_descr) AS sub_ft_count, SUM(actual_area) AS gross_sf, SUM(effec_area) AS effec_sf
    FROM Subarea
    GROUP BY RE, building)
    USING (RE, building)
    /* sub5 */
    LEFT JOIN
    (SELECT RE, SUM(actual_area) AS parcel_bldng_sf
    FROM Subarea
    GROUP BY RE)
    USING (RE)
    )
    WHERE property_use='0100' AND type_descr LIKE '%SFR%'
    """
    )
aggregates = pd.DataFrame(aggregates.fetchall(),
                                columns=['RE', 'building', 'bldng_count', 'land_value', 'feature_count',
                                         'avg_ft_grade', 'features_value', 'sub_ft_count', 'gross_sf',
                                         'effec_sf', 'parcel_bldng_sf'])


owners = con.execute(
    """
    SELECT 
        RE, building, owner
    FROM Parcel
    LEFT JOIN Building 
    USING (RE)
    LEFT JOIN Owner 
    USING (RE)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    GROUP BY RE, building
    """
    )
owners = pd.DataFrame(owners.fetchall(),
                               columns=['RE', 'building', 'owner'])

fts = con.execute(
    """
    SELECT 
        RE, building, ft_descr
    FROM Parcel
    LEFT JOIN Building 
    USING (RE)
    LEFT JOIN Feature
    USING (RE, building)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    """
    )
housing_features = pd.DataFrame(fts.fetchall(),
                               columns=['RE', 'building', 'ft_descr'])

rms = con.execute(
    """
    SELECT RE, building, baths, bedrooms, stories, rooms
    FROM Parcel
    LEFT JOIN Building
    USING (RE)
    LEFT JOIN
    (SELECT  
        RE, building, SUM(baths) AS baths, SUM(bedrooms) AS bedrooms, SUM(stories) AS stories, 
        SUM(rooms) AS rooms
    FROM 
        (SELECT	
            RE, building,
            CASE WHEN structure_descr = 'Baths' THEN units
                ELSE 0 END AS baths,
            CASE WHEN structure_descr = 'Bedrooms' THEN units
                ELSE 0 END AS bedrooms,
            CASE WHEN structure_descr = 'Stories' THEN units
                ELSE 0 END AS stories,
            CASE WHEN structure_descr = 'Rooms / Units' THEN units
                ELSE 0 END AS rooms
        FROM Utility)
        GROUP BY RE, building)
    USING (RE, building)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    """
    )
housing_rooms = pd.DataFrame(rms.fetchall(),
                               columns=['RE', 'building', 'baths', 'bedrooms', 'stories',
                                       'rooms'])

dists = con.execute(
    """
    SELECT
        RE, char_descr AS district
    FROM Parcel
    LEFT JOIN Building
    USING (RE)
    LEFT JOIN Character
    USING (RE)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    """
    )
districts = pd.DataFrame(dists.fetchall(),
                               columns=['RE', 'district'])

land = con.execute(
    """
    SELECT
        RE, building, use_descr AS primary_land_use, MAX(units) AS max_units, 
        COUNT(use_descr) AS land_use_count
    FROM Parcel
    LEFT JOIN Building
    USING (RE)
    LEFT JOIN Common
    USING (RE)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    GROUP BY RE, building
    """
    )
land_uses = pd.DataFrame(land.fetchall(),
                               columns=['RE', 'building', 'primary_land_use', 'max_units', 
                                        'land_use_count'])

sub = con.execute(
    """
    SELECT 
        RE, building, substructure_descr
    FROM Parcel
    LEFT JOIN Building 
    USING (RE)
    LEFT JOIN Subarea
    USING (RE, building)
    WHERE property_use='0100' AND type_descr LIKE '%SFR%' AND substructure_descr <> 'Base Area'
    """
    )
subarea = pd.DataFrame(sub.fetchall(),
                               columns=['RE', 'building', 'substructure_descr'])

site = con.execute(
    """
    SELECT
        Parcel.RE, street_num, direction, street_name, street_type, unit, Site.city, Site.zipcode, 
        building_num
    FROM Parcel
    LEFT JOIN Building
    USING (RE)
    LEFT JOIN Site
    ON Parcel.RE = Site.RE AND Building.building = Site.building_num
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%'
    GROUP BY Parcel.RE, Building.building
    """
    )
sites = pd.DataFrame(site.fetchall(),
                               columns=['RE', 'street_num', 'direction', 'street_name', 'street_type',
                                       'unit', 'city', 'zipcode', 'building'])



con.close()


In [3]:
def most_common(df, col_name):
    """Gets 10 most common categorical variables in dataframe column. Changes all
    other variables to 'Other'.

    Parameters
    ----------
    df : DataFrame
        Pandas dataframe
    col_name : str
        Name of column in dataframe

    Returns
    -------
    DataFrame
        Pandas series with variables outside of 10 most common changed to 'Other'
    """
    counts = df[col_name].value_counts()
    top_10 = list(counts[:10].index)
    new_col = []
    for thing in df[col_name]:
        if thing in top_10:
            new_col.append(thing)
        else:
            new_col.append(None)
    df[col_name] = new_col
    print(df[col_name].value_counts())

In [4]:
def get_dummies(df, col_name, prefix, groupby_list):
    """Gets dummy variables of 10 most common variables and 'Other'

    Parameters
    ----------
    df : DataFrame
        Pandas dataframe
    col_name : str
        Name of column in dataframe
    prefix : str
        Prefix used in dummy variable column names
    groupby_list : list
        List of column names to group df by before getting sums for merged dataframe

    Returns
    -------
    DataFrame
        Pandas series with variables outside of 10 most common changed to 'Other'
    """
    most_common(df, col_name)
    df2 = pd.get_dummies(df, columns=[col_name], prefix=prefix)
    df2 = df2.groupby(by=groupby_list).max().reset_index()
    return df2

In [5]:
housing_features = get_dummies(housing_features, 'ft_descr', 'ft', ['RE', 'building'])

Firep Prf     74639
Pool          29490
Firep Ms      29431
Scr Porch     27286
Cov Patio     21775
Deck Wd       21450
Shed wood     12942
Firep Gas     12097
Carport Al    11628
Screen En     11172
Name: ft_descr, dtype: int64


In [6]:
districts['district'] = [x.title() if x != None else None for x in districts['district']]
districts_final = get_dummies(districts, 'district', 'd', ['RE'])

Lake                                          4788
Local Riverside Avondale Historic District    3814
St Johns River                                1671
Golf Course                                   1223
Local Springfield Historic District           1221
Canal                                         1190
Miscellaneous Waterway                         770
Trout River                                    531
National Register Ortega Historic District     446
Ocean Front                                    317
Name: district, dtype: int64


In [7]:
land_uses.drop(columns=['max_units'], inplace=True)
landuses_final = get_dummies(land_uses, 'primary_land_use', 'land', ['RE', 'building'])

RES LD 3-7 UNITS PER AC             178834
RES POND LD 3-7 UNITS PER AC         21391
RES MD 8-19 UNITS PER AC             16190
RES NATURAL LD 3-7 UNITS PER AC      14421
RES RURAL 2 OR LESS UNITS PER AC      5536
RES RIVER LD 3-7 UNITS PER AC         3343
RES CANAL LD 3-7 UNITS PER AC         3164
RES GOLF LD 3-7 UNITS PER AC          2088
RES MARSH LD 3-7 UNITS PER AC         1167
COMM/RES/OFF                          1145
Name: primary_land_use, dtype: int64


In [8]:
subarea_final = get_dummies(subarea, 'substructure_descr', 'sub', ['RE','building'])

Finished Open Porch       276037
Finished Garage           142509
Addition                  100687
Finished upper story 1     51299
Fin Screened Porch         42020
Unfinished Storage         41863
Unfin Open Porch           27342
Unfinished Garage          23814
Finished Encl Porch        20945
Finished Carport           16193
Name: substructure_descr, dtype: int64


In [9]:
df = pd.merge(housing_inventory, housing_features, on=['RE', 'building'])
df = df.merge(aggregates, how='left', on=['RE', 'building'])
df = df.merge(owners, how='left', on=['RE', 'building'])
df = df.merge(housing_rooms, how ='left', on=['RE', 'building'])
df = df.merge(districts_final, how='left', on='RE')
df = df.merge(landuses_final, how='left', on=['RE','building'])
df = df.merge(subarea_final, how='left', on=['RE','building'])
df = df.merge(sites, how='left', on=['RE', 'building'])
df.head()

,RE,section,township,range,tile,mailing_address_1,mailing_address_2,mail_city,mail_state,mail_zipcode,...,sub_Unfin Open Porch,sub_Unfinished Garage,sub_Unfinished Storage,street_num,direction,street_name,street_type,unit,city,zipcode
0,0000060030R,01,2S,23E,3401,2503 SUMMERFIELD LN,None,JACKSONVILLE,FL,32234,...,0.0,0.0,0.0,2503,None,SUMMERFIELD,LN,None,JACKSONVILLE,32234
1,0000070010R,01,2S,23E,3401,C/O TAX DEPARTMENT,9540 SAN JOSE BLVD,JACKSONVILLE,FL,32257,...,0.0,0.0,0.0,2610,N,US 301,HWY,None,JACKSONVILLE,32234
2,0000090100R,12,2S,23E,3412,2495 U S 301 HWY N,None,BALDWIN,FL,32234,...,1.0,0.0,0.0,2495,N,US 301,HWY,None,JACKSONVILLE,32234
3,0000110000R,12,2S,23E,3412,2425 US HWY 301 N,None,BALDWIN,FL,32234,...,0.0,0.0,0.0,2435,N,US 301,HWY,None,JACKSONVILLE,32234
4,0000120010R,12,2S,23E,3412,2204 US HIGHWAY 301 N,None,JACKSONVILLE,FL,32234,...,0.0,0.0,0.0,2204,N,US 301,HWY,None,JACKSONVILLE,32234


In [10]:
df.shape

(252718, 91)

In [11]:
df.to_csv('D:/Data/JacksonvilleHouses.csv', index=False)

In [2]:
engine = create_engine('sqlite:///D:\\Data\\DuvalProperties.sqlite')
con = engine.connect()
sale_db = con.execute(
    """
    SELECT (substr(RE,1,6)||' '||substr(RE,7,4)) AS RE, sale_id, seller, sale_date, price,
        building, actual_yr_built, heated_sf, 
        CAST(substr(sale_date,1,4)AS INT) - actual_yr_built AS age_at_sale, neighborhood, Site.zipcode,
		Site.street_num||' '||IFNULL(Site.direction,'')||' '||Site.street_name||' '||Site.street_type AS site_address
    FROM Parcel
    LEFT JOIN Building USING (RE)
    LEFT JOIN Sale USING (RE)
    LEFT JOIN Qualification USING (q_id)
    LEFT JOIN Site USING (RE)
    WHERE property_use = '0100' AND type_descr LIKE '%SFR%' AND status = 'Qualified' AND improved = 'I'
        AND CAST(substr(sale_date,1,4) AS INT) > actual_yr_built
    GROUP BY or_bk, or_pg, RE
    """
    )
house_sales = pd.DataFrame(sale_db.fetchall(),
                                columns=['RE', 'sale_id', 'seller', 'sale_date', 'price',
                                        'building', 'actual_yr_built', 'heated_sf', 'age_at_sale',
                                        'neighborhood', 'zipcode','address'])
con.close()
house_sales.to_csv('D:/Data/HomeSales.csv', index=False)